In [3]:
#preparing for 4.2 modeling
#next is making the map after this 


In [ ]:
import pandas as pd
import pmdarima as pm
from pmdarima.arima import auto_arima
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context("paper", font_scale = 1.4)
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import math
import warnings
warnings.simplefilter("ignore")
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from matplotlib.pylab import rcParams
from statsmodels.tools.eval_measures import rmse
from statsmodels.tsa.holtwinters import ExponentialSmoothing

#no fbprophet import 

In [30]:
df3 = pd.read_csv('df2.csv')

In [31]:
df3.head()

,Unnamed: 0,Complaint_Date,Complaint_Time,Crime,Crime_Description,Offense,Borough,X_COORD_CD,Y_COORD_CD,Suspect_Age,Suspect_Gender,Patrol_Borough,Station_Name,Victim Age,Victim_Race,Victim_Gender,Complaint_Date_Time
0,0,06/09/2018,15:54:00,GRAND LARCENY,"LARCENY,GRAND FROM PERSON,PURS",FELONY,BRONX,1010257.0,248767.0,<18,M,PATROL BORO BRONX,TREMONT AVENUE,45-64,WHITE HISPANIC,F,2018-06-09 15:54:00
1,1,05/19/2018,12:25:00,FORGERY,"FORGERY,ETC.,UNCLASSIFIED-FELO",FELONY,MANHATTAN,990886.0,213048.0,45-64,M,PATROL BORO MAN SOUTH,42 ST.-GRAND CENTRAL,25-44,WHITE,F,2018-05-19 12:25:00
2,2,11/04/2018,00:00:00,DANGEROUS WEAPONS,WEAPONS POSSESSION 3,FELONY,MANHATTAN,980772.0,194845.0,25-44,M,PATROL BORO MAN SOUTH,SOUTH FERRY,65+,BLACK,M,2018-11-04 00:00:00
3,3,01/18/2018,12:23:00,POSSESSION OF STOLEN PROPERTY,"STOLEN PROPERTY 3,POSSESSION",MISDEMEANOR,MANHATTAN,989294.0,215445.0,25-44,M,PATROL BORO MAN SOUTH,47-50 STS./ROCKEFELLER CTR.,25-44,WHITE,M,2018-01-18 12:23:00
4,4,12/29/2018,21:30:00,JOSTLING,JOSTLING,MISDEMEANOR,BRONX,1004749.0,240880.0,45-64,F,PATROL BORO BRONX,161 ST.-YANKEE STADIUM,25-44,BLACK HISPANIC,M,2018-12-29 21:30:00


In [32]:
df3 = df3.set_index('Complaint_Date_Time')

In [ ]:
#ARIMA Model/ SARIMAX 

In [33]:
# make a copy of crimes dataset to work on, so we don't mess up the original dataset 
df3_copy = df3.copy()
df3_copy.head()

,Unnamed: 0,Complaint_Date,Complaint_Time,Crime,Crime_Description,Offense,Borough,X_COORD_CD,Y_COORD_CD,Suspect_Age,Suspect_Gender,Patrol_Borough,Station_Name,Victim Age,Victim_Race,Victim_Gender
Complaint_Date_Time,,,,,,,,,,,,,,,,
2018-06-09 15:54:00,0,06/09/2018,15:54:00,GRAND LARCENY,"LARCENY,GRAND FROM PERSON,PURS",FELONY,BRONX,1010257.0,248767.0,<18,M,PATROL BORO BRONX,TREMONT AVENUE,45-64,WHITE HISPANIC,F
2018-05-19 12:25:00,1,05/19/2018,12:25:00,FORGERY,"FORGERY,ETC.,UNCLASSIFIED-FELO",FELONY,MANHATTAN,990886.0,213048.0,45-64,M,PATROL BORO MAN SOUTH,42 ST.-GRAND CENTRAL,25-44,WHITE,F
2018-11-04 00:00:00,2,11/04/2018,00:00:00,DANGEROUS WEAPONS,WEAPONS POSSESSION 3,FELONY,MANHATTAN,980772.0,194845.0,25-44,M,PATROL BORO MAN SOUTH,SOUTH FERRY,65+,BLACK,M
2018-01-18 12:23:00,3,01/18/2018,12:23:00,POSSESSION OF STOLEN PROPERTY,"STOLEN PROPERTY 3,POSSESSION",MISDEMEANOR,MANHATTAN,989294.0,215445.0,25-44,M,PATROL BORO MAN SOUTH,47-50 STS./ROCKEFELLER CTR.,25-44,WHITE,M
2018-12-29 21:30:00,4,12/29/2018,21:30:00,JOSTLING,JOSTLING,MISDEMEANOR,BRONX,1004749.0,240880.0,45-64,F,PATROL BORO BRONX,161 ST.-YANKEE STADIUM,25-44,BLACK HISPANIC,M


In [34]:
#drops race and gender because that's not changing over time. Give model cols that will be predictive of
#the crimes that will happen...races of people don't change over time. 

# drop columns
df3_copy = df3_copy.drop(['Suspect_Gender', 'Victim_Race', 'Victim_Gender', 
                         'Complaint_Time', 'Unnamed: 0', 'Crime_Description', 'Offense',
                         'X_COORD_CD', 'Y_COORD_CD', 'Suspect_Age', 'Station_Name'], axis = 1)

In [35]:
df3_copy.head()

,Complaint_Date,Crime,Borough,Patrol_Borough,Victim Age
Complaint_Date_Time,,,,,
2018-06-09 15:54:00,06/09/2018,GRAND LARCENY,BRONX,PATROL BORO BRONX,45-64
2018-05-19 12:25:00,05/19/2018,FORGERY,MANHATTAN,PATROL BORO MAN SOUTH,25-44
2018-11-04 00:00:00,11/04/2018,DANGEROUS WEAPONS,MANHATTAN,PATROL BORO MAN SOUTH,65+
2018-01-18 12:23:00,01/18/2018,POSSESSION OF STOLEN PROPERTY,MANHATTAN,PATROL BORO MAN SOUTH,25-44
2018-12-29 21:30:00,12/29/2018,JOSTLING,BRONX,PATROL BORO BRONX,25-44


In [36]:
df3_copy = df3_copy.drop(['Patrol_Borough', 'Victim Age', 'Complaint_Date'], axis = 1)

In [37]:
df3_copy.head()

,Crime,Borough
Complaint_Date_Time,,
2018-06-09 15:54:00,GRAND LARCENY,BRONX
2018-05-19 12:25:00,FORGERY,MANHATTAN
2018-11-04 00:00:00,DANGEROUS WEAPONS,MANHATTAN
2018-01-18 12:23:00,POSSESSION OF STOLEN PROPERTY,MANHATTAN
2018-12-29 21:30:00,JOSTLING,BRONX


In [38]:
# count number of crimes by borough
df3_copy = df3_copy.groupby(['Borough'], as_index = True).resample('M').agg({"Crime":'size'})
# rename crime column
df3_copy.rename(columns = {'Crime':'Crime_Number'}, inplace = True)
# view data
df3_copy.head()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'